In [2]:
# library
import numpy as np
import pandas as pd
import plotly.express as px

import json

site_list = ["bbc", "cnn", "foxnews", "nationalreview", "washingtontimes", "newsweek"]

In [3]:
def open_json(file_path):
    data = []
    with open(file_path, "r") as f:
        for line in f:
            try:
                # Load each line as a separate JSON object
                json_object = json.loads(line)
                data.append(json_object)
            except json.JSONDecodeError:
                # Handle possible json decoding errors
                continue

    # Creating a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    
    # Selecting specific columns to include in the final DataFrame
    df_json = df[["category", "headline", "link", "short_description", "date"]]
    return df_json
    # with open(file_path, "r") as f:
    #     data = json.load(f)
    # # Ensuring each entry is a dictionary and extracting them directly
    # data = [entry for entry in data if isinstance(entry, dict)]
    
    # # Creating a DataFrame from the list of dictionaries
    # df = pd.DataFrame(data)
    
    # # Selecting specific columns to include in the final DataFrame
    # df_json = df[["category", "headline", "link", "short_description", "date"]]
    # return df_json

In [18]:
def open_article_json(file_path):
    data = []
    with open(file_path, "r") as f:
        for line in f:
            try:
                # Load each line as a separate JSON object
                json_object = json.loads(line)
                data.append(json_object)
            except json.JSONDecodeError:
                # Handle possible json decoding errors
                continue

    df = pd.DataFrame(data)

    df_json = df[["title","authors","url","date_publish","description","maintext","wayback_time","text_len","year_week","week_start_date"]]
    # df_json.describe()
    return df_json

In [4]:
def cleanDF(df_json):
    # clean the data
    # drop the main text that is empty
    df_json = df_json.dropna(subset=["category", "headline", "short_description"])

    # drop the duplicates
    df_json = df_json.drop_duplicates(subset=["category", "headline", "short_description"])

    # add a new column based on the length of the main text
    df_json["headline_len"] = df_json["headline"].apply(lambda x: len(x))

    # add a new column based on the length of the main text
    df_json["short_description_len"] = df_json["short_description"].apply(lambda x: len(x))

    # sort
    df_json = df_json.sort_values(by="date", ascending=True)

    return df_json

In [5]:
file_path_json = "data/News_Category_Dataset_v3.json"
df_json = open_json(file_path_json)
df_json = cleanDF(df_json)

df_json.to_json("data/articles_all/words_category.json", orient='records', lines=True)

In [6]:
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS

df_temp = df_json[["headline", "short_description", "category"]].copy()

#only keep the crime category
df_crime = df_temp[df_temp["category"] == "CRIME"]
df_crime.dropna(inplace=True)

# parse the title and description into lower case
df_crime["headline"] = df_crime["headline"].str.lower()
df_crime["short_description"] = df_crime["short_description"].str.lower()

# remove the punctuation
df_crime["headline"] = df_crime["headline"].str.replace(r'[^\w\s]','')
df_crime["short_description"] = df_crime["short_description"].str.replace(r'[^\w\s]','')

# remove the stop words
df_crime["headline"] = df_crime["headline"].apply(lambda x: " ".join([word for word in x.split() if word not in STOP_WORDS]))
df_crime["short_description"] = df_crime["short_description"].apply(lambda x: " ".join([word for word in x.split() if word not in STOP_WORDS]))

# remove the numbers
df_crime["headline"] = df_crime["headline"].str.replace(r'\d+', '')
df_crime["short_description"] = df_crime["short_description"].str.replace(r'\d+', '')

#only keep the science category
df_science = df_temp[df_temp["category"] == "SCIENCE"]
df_science.dropna(inplace=True)

# parse the title and description into lower case
df_science["headline"] = df_science["headline"].str.lower()
df_science["short_description"] = df_science["short_description"].str.lower()

# remove the punctuation
df_science["headline"] = df_science["headline"].str.replace(r'[^\w\s]','')
df_science["short_description"] = df_science["short_description"].str.replace(r'[^\w\s]','')

# remove the stop words
df_science["headline"] = df_science["headline"].apply(lambda x: " ".join([word for word in x.split() if word not in STOP_WORDS]))
df_science["short_description"] = df_science["short_description"].apply(lambda x: " ".join([word for word in x.split() if word not in STOP_WORDS]))

# remove the numbers
df_science["headline"] = df_science["headline"].str.replace(r'\d+', '')
df_science["short_description"] = df_science["short_description"].str.replace(r'\d+', '')

/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/2010804212.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crime.dropna(inplace=True)
/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/2010804212.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crime["headline"] = df_crime["headline"].str.lower()
/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/2010804212.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [7]:
from collections import Counter

# Function to count word frequencies
def count_word_frequencies(text):
    # Split the text into words and count the occurrences of each word
    word_counts = Counter(text.split())
    return word_counts

In [8]:
# Apply the function to the title and description columns
df_crime["headline_word_counts"] = df_crime["headline"].apply(count_word_frequencies)
df_crime["short_description_word_counts"] = df_crime["short_description"].apply(count_word_frequencies)
print(df_crime[["headline", "headline_word_counts", "short_description", "short_description_word_counts"]].head())

                                                 headline  \
208213  karen swift's funeral planned saturday homicid...   
208133  tim cole, convict exonerated death, gets texas...   
208134       subject gun control, government wins justice   
207545  hannah kelly, pastor's daughter, dies accident...   
207481  george huguely murder trial: jurors deliberate...   

                                     headline_word_counts  \
208213  {'karen': 1, 'swift's': 1, 'funeral': 1, 'plan...   
208133  {'tim': 1, 'cole,': 1, 'convict': 1, 'exonerat...   
208134  {'subject': 1, 'gun': 1, 'control,': 1, 'gover...   
207545  {'hannah': 1, 'kelly,': 1, 'pastor's': 1, 'dau...   
207481  {'george': 1, 'huguely': 1, 'murder': 1, 'tria...   

                                        short_description  \
208213  police released cause death case confirmed inv...   
208133  legislature created timothy cole advisory pane...   
208134  i'm advocate gun control, knee-jerk opponent n...   
207545  20-year-old ha

/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/1607594776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crime["headline_word_counts"] = df_crime["headline"].apply(count_word_frequencies)
/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/1607594776.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crime["short_description_word_counts"] = df_crime["short_description"].apply(count_word_frequencies)


In [9]:
# Apply the function to the title and description columns
df_science["headline_word_counts"] = df_science["headline"].apply(count_word_frequencies)
df_science["short_description_word_counts"] = df_science["short_description"].apply(count_word_frequencies)
print(df_science[["headline", "headline_word_counts", "short_description", "short_description_word_counts"]].head())

                                                 headline  \
209489  russian cargo ship docks international space s...   
209490  robots play catch, starring agile justin rolli...   
209491   thomas edison voted iconic inventor u.s. history   
209492  aurora borealis caused huge solar storm shimme...   
209427  space shuttle challenger disaster marks 26th a...   

                                     headline_word_counts  \
209489  {'russian': 1, 'cargo': 1, 'ship': 1, 'docks':...   
209490  {'robots': 1, 'play': 1, 'catch,': 1, 'starrin...   
209491  {'thomas': 1, 'edison': 1, 'voted': 1, 'iconic...   
209492  {'aurora': 1, 'borealis': 1, 'caused': 1, 'hug...   
209427  {'space': 1, 'shuttle': 1, 'challenger': 1, 'd...   

                                        short_description  \
209489  gallery: space station's expedition 30 mission...   
209490  image 1: throw hizook reports, dlr started rol...   
209491  doesn't mean jobs lacks fans wake death year. ...   
209492  aurora boreali

/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/641472430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science["headline_word_counts"] = df_science["headline"].apply(count_word_frequencies)
/var/folders/fr/tgw5syk508j35cyfwv4bhgq00000gn/T/ipykernel_33864/641472430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science["short_description_word_counts"] = df_science["short_description"].apply(count_word_frequencies)


In [10]:
from collections import Counter

# Define a set of non-related words
non_related_words = {
    'new', 'florida', 'texas', 'nyc', 'california', 'says', '2', 'idaho', 'chicago',
    'georgia', 'carolina', 'york', 'city', 'video', 'car', 'home', 'years', 'family',
    'los', 'mom', 'angeles', 'north', 'men', 'state', 'judge', 'officials', '4', 'house',
    'near', 'people', 'sex', 'virginia', 'university', 'court', 'philadelphia', 'fire',
    'subway', 'nypd', '1', 'arizona', 'washington', 'women', 'school', 'teen', '1', '2', '3',
    'mother', 'woman', 'shows', 'girl', 'driver', 'student', 'child', 'search', 'leaves', 'found',
    'man', ':', ',', '(video)', '&', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
    '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
    '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
    '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
    '-', 'father', 'brother', 'sister', 'son', 'daughter', 'husband', 'wife', 'boyfriend', 'girlfriend',
    'boy', '--', '(photo)', '(photos)', 'it\'s', 'don\'t', 'can\'t', 'won\'t', 'didn\'t', 'doesn\'t',
    'i\'m', 'you\'re', 'he\'s', 'she\'s', 'we\'re', 'they\'re', 'i\'ll', 'you\'ll', 'he\'ll', 'she\'ll',
    'we\'ll', 'they\'ll', 'i\'ve', 'you\'ve', 'we\'ve', 'they\'ve', 'i\'d', 'you\'d', 'he\'d', 'she\'d',
    'we\'d', 'they\'d', 'i\'ll', 'you\'ll', 'he\'ll', 'she\'ll', 'we\'ll', 'they\'ll', 'i\'ve', 'you\'ve',
    'we\'ve', 'they\'ve', 'i\'d', 'you\'d', 'he\'d', 'she\'d', 'we\'d', 'they\'d', 'i\'ll', 'you\'ll',
    'he\'ll', 'she\'ll', 'we\'ll', 'they\'ll', 'i\'ve', 'you\'ve', 'we\'ve', 'they\'ve', 'i\'d', 'you\'d',
    'he\'d', 'she\'d', 'we\'d', 'they\'d', 'i\'ll', 'you\'ll', 'he\'ll', 'she\'ll', 'we\'ll', 'they\'ll',
    'i\'ve', 'you\'ve', 'we\'ve', 'they\'ve', 'i\'d', 'you\'d', 'he\'d', 'she\'d', 'we\'d', 'they\'d',
    'i\'ll', 'you\'ll', 'he\'ll', 'she\'ll', 'we\'ll', 'they\'ll', 'i\'ve', 'you\'ve', 'we\'ve', 'they\'ve',
    'i\'d', 'you\'d', 'he\'d', 'she\'d', 'we\'d', 'they\'d', 'i\'ll', 'you\'ll', 'he\'ll', 'she\'ll',
    'we\'ll', 'they\'ll', 'i\'ve', 'you\'ve', 'we\'ve', 'they\'ve', 'i\'d', 'you\'d', 'he\'d', 'she\'d',
    'we\'d', 'they\'d', 'i\'ll', 'you\'ll', 'he\'ll', 'she\'ll', 'we\'ll', 'they\'ll', 'i\'ve', 'you\'ve',
    'san', 'high', 'u.s.', 'ohio', 'photo'
}

# Combine all word counts from titles and descriptions
all_word_counts_crime = Counter()
for index, row in df_crime.iterrows():
    all_word_counts_crime.update(row["headline_word_counts"])
    # all_word_counts.update(row["description_word_counts"])

# Get the top 50 most common words
top_50_crime_words = all_word_counts_crime.most_common(50)
top_100_crime_words = all_word_counts_crime.most_common(100)

top_crime_words_filtered = [(word, count) for word, count in top_100_crime_words if word not in non_related_words]

# Print the filtered list
print("Top words related to crime:")
for word, count in top_crime_words_filtered:
    print(f"{word}: {count}")

# Combine all word counts from titles and descriptions
all_word_counts_science = Counter()
for index, row in df_science.iterrows():
    all_word_counts_science.update(row["headline_word_counts"])
    # all_word_counts.update(row["description_word_counts"])

# Get the top 50 most common words
top_50_science_words = all_word_counts_science.most_common(50)
top_100_science_words = all_word_counts_science.most_common(100)

# Filter the top words list to exclude non-crime-related words

top_science_words_filtered = [(word, count) for word, count in top_100_science_words if word not in non_related_words]

# Print the filtered list
print("Top words related to science:")
for word, count in top_science_words_filtered:
    print(f"{word}: {count}")

# # Print the top 50 words with their counts
# print("Top 50 most frequent words:")
# for word, count in top_50_words:
#     print(f"{word}: {count}")

# # Print the top 100 words with their counts
# print("Top 100 most frequent words:")
# for word, count in top_100_words:
#     print(f"{word}: {count}")

Top words related to crime:
police: 456
shooting: 254
cops: 184
killed: 174
allegedly: 165
accused: 161
arrested: 151
suspect: 148
death: 144
shot: 134
dead: 126
murder: 119
cop: 107
charged: 105
officer: 102
killing: 102
prison: 87
missing: 80
gun: 79
alleged: 64
dies: 62
guilty: 60
shooter: 60
case: 57
fatally: 57
trial: 55
attack: 54
officers: 54
injured: 51
charges: 50
kills: 49
shoots: 46
gets: 45
body: 44
arrest: 43
fbi: 41
black: 41
suspected: 40
sentenced: 39
killer: 39
life: 38
kill: 38
dead,: 38
rape: 37
crime: 34
fatal: 34
caught: 33
convicted: 33
assault: 32
gunman: 32
deadly: 32
faces: 31
face: 31
Top words related to science:
scientists: 154
space: 144
study: 125
nasa: 105
science: 100
earth: 70
mars: 70
here's: 60
suggests: 59
like: 53
moon: 48
ancient: 48
brain: 45
solar: 41
planet: 41
research: 39
life: 39
way: 37
help: 36
world: 35
photos: 34
watch:: 33
time: 32
nasa's: 32
discovered: 32
rocket: 31
climate: 31
big: 29
mission: 29
change: 29
star: 27
watch: 27
alien: 2

In [11]:
# Load English tokenizer, tagger, 
# parser, NER and word vectors 
nlp = spacy.load("en_core_web_sm") 

# process top_words_filtered into a string
top_crime_words = [word for word, count in top_crime_words_filtered]
top_crime_words_str = " ".join(top_crime_words)

top_science_words = [word for word, count in top_science_words_filtered]
top_science_words_str = " ".join(top_science_words)

doc_crime = nlp(top_science_words_str)
doc_science = nlp(top_crime_words_str)
    
crime_words = [token.lemma_ for token in doc_crime]
print(crime_words, len(crime_words))

science_words = [token.lemma_ for token in doc_science]
print(science_words, len(science_words))

['scientist', 'space', 'study', 'nasa', 'science', 'earth', 'mar', 'here', "'s", 'suggest', 'like', 'moon', 'ancient', 'brain', 'solar', 'planet', 'research', 'life', 'way', 'help', 'world', 'photo', 'watch', ':', 'time', 'nasa', "'s", 'discover', 'rocket', 'climate', 'big', 'mission', 'change', 'star', 'watch', 'alien', 'find', 'sky', 'human', 'neil', 'red', 'rover', 'spacecraft', 'mystery', 'asteroid', 'researcher', 'black', 'astronomer', 'telescope', 'meteor', 'astronaut', 'mysterious', 'universe', 'human', 'planet', 'launch', 'look', 'degrasse', 'station', 'scientific', 'water', 'spacex', 'giant', 'light', 'actually', 'sea', 'world', "'s", 'make', 'astronaut', 'well', 'evolution', 'eclipse', 'night', 'amazing', 'reveal', 'tyson', 'reveal', 'face', 'day', 'look', 'huge', 'suggest', 'sun', 'galaxy', 'curiosity', 'future', 'rare'] 88
['police', 'shoot', 'cop', 'kill', 'allegedly', 'accuse', 'arrest', 'suspect', 'death', 'shoot', 'dead', 'murder', 'cop', 'charge', 'officer', 'kill', 'p

In [12]:
# scan through the headline and short description to see if the words are in the top words
def scan_words(text, words):
    text = text.split()
    for word in text:
        if word in words:
            return True
    return False

In [23]:
# open the file in articles_all one by one to scan their words in title and description

for site in site_list:
    file_path = f"data/articles_all/articles{site}.json"
    df = open_article_json(file_path)

    # Handle None by replacing with empty string
    df['title'] = df['title'].fillna('').str.lower().str.replace(r'[^\w\s]', '').str.replace(r'\d+', '')
    df['description'] = df['description'].fillna('').str.lower().str.replace(r'[^\w\s]', '').str.replace(r'\d+', '')

    # Remove stop words and numbers, streamlined to avoid repetition
    for column in ['title', 'description']:
        df[column] = df[column].apply(lambda x: " ".join([word for word in x.split() if word not in STOP_WORDS]))

    def check_top_words(row, top_words):
        title_words = row["title"].split()
        description_words = row["description"].split()
        total_words = title_words + description_words
        count = sum(1 for word in total_words if word in top_words)
        return count > 2

    # Assigning categories based on presence of top words
    df["category"] = "Others"
    df.loc[df.apply(lambda row: check_top_words(row, top_crime_words), axis=1), "category"] = "Crime"
    df.loc[df.apply(lambda row: check_top_words(row, top_science_words), axis=1), "category"] = "Science"
    df.loc[df.apply(lambda row: check_top_words(row, top_crime_words) and check_top_words(row, top_science_words), axis=1), "category"] = "Both"

    # Print the updated DataFrame
    print(df)

    #output the file as a json file
    df.to_json(f"data/articles_all/articles{site}_category.json", orient='records', lines=True)
    


                                                   title authors  \
0            killed dozens injured oregon tour bus crash      []   
1              george h w bush intensive care 'setbacks'      []   
2                    hillary clinton hospital blood clot      []   
3                                               election      []   
4                       voters react 'fiscal cliff' deal      []   
...                                                  ...     ...   
10846  alabama mother: joy babies born mum rare doubl...      []   
10847   dizzying video shows cliffside rescue california      []   
10848         expert: ai ‘hurts ability democracy alive’      []   
10849                          what’s taylor swift 2024?      []   
10850  dangerous high surf pummels california coast a...      []   

                                                     url         date_publish  \
0       http://www.bbc.com/news/world-us-canada-20879173  2012-12-31 18:06:09   
1       http://www.bb